Import Tensorflow and open a session.

In [2]:
%run 'Ising.py'

### Determinant
Calculate matrix determinant

In [4]:
W = tf.constant([[0.,1.,1.,1.],
                 [-1.,0.,1.,-1.],
                 [-1.,-1.,0.,1.],
                 [-1.,1.,-1.,0.]])
f = tf.matrix_determinant(W)
tf.Session().run(f)

9.0

Train by minimizing the ln(det(W)). TF knows how to do the gradient.

In [5]:
W = tf.Variable([[0.,1.,1.,1.],
                 [-1.,0.,1.,-1.],
                 [-1.,-1.,0.,1.],
                 [-1.,1.,-1.,0.]])
f = tf.log(tf.matrix_determinant(W))
init = tf.global_variables_initializer()
optimizer = tf.train.GradientDescentOptimizer(0.1)
train = optimizer.minimize(f)
sess = tf.Session()
sess.run(init)
for i in range(5):
    sess.run(train)
    print(sess.run(f))

2.06162
1.91632
1.75986
1.5904
1.4056


### Sparse Tensor
Sparse tensor must be converted to dense when passed to the determinant op.

In [337]:
W = tf.SparseTensor(indices=[[0, 0], [1, 1]], values=[1., 2.], dense_shape=[2, 2])
W_dense = tf.sparse_tensor_to_dense(W)
f = tf.log(tf.matrix_determinant(W_dense))
sess.run(f)

0.69314718

## Entanglement
### Entanglement Regions

In [43]:
list(Region({1,2},4).equivalences())

[{2, 3}, {0, 1}, {1, 2}, {0, 3}]

In [44]:
Region({1,2},4).config()

array([ 1.000, -1.000, -1.000,  1.000])

In [42]:
list(Regions(8).consecutives())

[set(),
 {0},
 {0, 1},
 {0, 1, 2},
 {0, 1, 2, 3},
 {0, 1, 2, 3, 4},
 {0, 1, 2, 3, 4, 5},
 {0, 1, 2, 3, 4, 5, 6},
 {0, 1, 2, 3, 4, 5, 6, 7}]

In [41]:
list(Regions(8).representatives())

[set(),
 {0},
 {0, 1},
 {0, 2},
 {0, 3},
 {0, 4},
 {0, 1, 2},
 {0, 1, 3},
 {0, 1, 4},
 {0, 2, 4},
 {0, 2, 5},
 {0, 1, 2, 3},
 {0, 1, 2, 4},
 {0, 1, 2, 5},
 {0, 1, 3, 5},
 {0, 1, 3, 6},
 {0, 1, 4, 5},
 {0, 2, 4, 6}]

### Entanglement Entropy
Entanglement entropy of free fermion.
$$H=\sum_{i}(1+(-)^i m)\mathrm{i}\chi_{i}\chi_{i+1}$$

In [745]:
system = FreeFermion(mass=1.e-10, size=8)
system.S([0,1,2,3,4])

0.84523244468672853

### Test

In [709]:
%run 'Ising.py'
machine = EFL(FreeFermion(mass=1.e-10, size=256), CylindricalModel(Lx = 8, Lz = 2))
training_set = machine.training_set('representatives')
writer = machine.writer()
writer.add_graph(sess.graph)
sess.run(machine.model.initialize)

In [712]:
for i in range(2000):
    machine.step += 1
    sess.run(machine.model.train, training_set)
    writer.add_summary(sess.run(machine.model.summarize, training_set), machine.step)
sess.run([machine.model.MSE, machine.model.L2, machine.model.cost], training_set)

[7.0578566877442751e-06, 0.0, 7.0578566877442751e-06]

### Square Lattice Adjacency Matices

In [130]:
%run 'Ising.py'
latt = SSLattice(width = 2, depth = 3)
latt.print_structure(upto = 'cell')

depth =  3
Chain[0]
├ Cell[0, 0] IR
├ Cell[0, 1] IR
Chain[1]
├ Cell[1, 0] UV
├ Cell[1, 1] IR
├ Cell[1, 2] UV
├ Cell[1, 3] IR
Chain[2]
├ Cell[2, 0] UV
├ Cell[2, 1] UV


### Test

In [345]:
%run 'Ising.py'

In [148]:
%run 'Ising.py'
machine = EFL(FreeFermion(mass=1.e-10, size=128), IsingModel(SSLattice(width = 8, depth = 3)))
machine.initialize()

In [149]:
machine.train(1000)

(0.00039590149887340286, array([ 1.300,  0.376,  0.075, -0.005]))

In [146]:
machine.data.source[machine.model.label] - machine.session.run(machine.model.prediction, machine.data.source)

array([ 0.007, -0.000,  0.025, -0.007, -0.017,  0.029,  0.010, -0.043,
        0.017, -0.008,  0.044,  0.011, -0.031, -0.014,  0.002, -0.094,
        0.031])

In [147]:
list(machine.model.variable.eval(machine.session))

[1.1014618119225039,
 0.30602383710755793,
 0.70192455586430091,
 -0.47729550035900731]

In [89]:
print(machine.session.run([machine.model.F,machine.model.F0],machine.data.source))

[array([-16.213, -15.695, -15.339, -15.236, -15.217, -15.526, -14.955,
       -14.770, -14.525, -14.424, -15.492, -14.867, -14.697, -14.269,
       -14.319, -14.450, -13.977]), -17.226283845371071]


In [100]:
a= tf.Variable([0.,0.,0.,0.,0.])
a0 = a[0]
ar = a[1:] 

In [104]:
ar

<tf.Tensor 'strided_slice_2:0' shape=(4,) dtype=float32>